In [162]:
import numpy as np
import matplotlib.pyplot as plt


In [163]:
# Initialize position and velocity of particles
pos= np.zeros((3,1000))
np.random.seed(0)
for j in range(100):
   for i in range(10):
      pos[0,j+100*i:j+100*(i+1)-1]= i%10
for j in range(10):
   for i in range(100):
      pos[1,j+10*i:j+10*(i+1)-1]=i%10
for i in range(1000):
    pos[2,i]=i%10

vel = np.random.rand(3, 1000)
vel=vel-0.5
vel=vel*176.8
# Constants
dt = 0.001
n_steps = 10000
sigma = 3.54
epsilon = 0.185
r_cutoff = 2.5*sigma
box_size = 1000


Function defined which is repeated in every loop to find distance between particles

In [165]:
#function to return distances
def get_dist(i):
    pos_copy = np.copy(pos)
    point = pos_copy[:,i,None]
    pos_dir = pos_copy - point
    dist = np.sqrt(np.sum(pos_dir**2,axis = 0))
    return dist, pos_dir


To store the distance between two particles and update the neighbours list


In [1]:
#updating array distances
distances = np.zeros((1000,1000))
distance_vector=np.zeros((3,1000,1000))
for i in range(1000):
    distances[i,:], distance_vector[:,:,i] = get_dist(i)
# neighbour_list = []
# neighbours=[]
# for i in range(1000):
#     for j in range(1000):    
#         if(distances[i][j]<=r_cutoff):
#             neighbours.append(j)
# neighbour_list.append(neighbours)

neighbours = distances < r_cutoff
friends = distances >= r_cutoff && distances < r_cutoff


SyntaxError: invalid syntax (2297521490.py, line 15)

Simulation loop begins

In [170]:
import time 
# Velocity Verlet algorithm
n_steps = 100
force = np.zeros((3, 1000))
force_new = np.zeros((3, 1000))
distances = distances + 0.00001
for step in range(n_steps):
    for i in range(1000):
        for j,bool in enumerate(list(neighbours[i,:])):
            if bool == False and i==j: continue
            force[:,i] += (24*epsilon*(2*(sigma**12/distances[i,j]**13) - (sigma**6/distances[i,j]**7)))*(distance_vector[:,i,j]/distances[i,j])

    abs_force = np.linalg.norm(force,axis=0)
    print(abs_force.min(),abs_force.max(),abs_force.mean(),abs_force.std())

    # Update positions
    pos += vel*dt + 0.5*force*dt**2
    #Check for wall crossings and handle them
    for k in range(1000):
        for j in range(3):
            if (pos[0][k] > box_size):
                pos[0][k] = pos[0][k]-box_size

            if (pos[1][k] > box_size):
                pos[1][k] = pos[0][k]-box_size

            if (pos[2][k] > box_size):
                pos[2][k] = pos[0][k]-box_size    

                if pos[0][k] < 0:
                    pos[0][k] = pos[0][k]+box_size

                if pos[1][k] < 0:
                    pos[1][k] = pos[1][k]+box_size  
                    
                if pos[2][k] < 0:
                    pos[2][k] = pos[2][k]+box_size  
        
    # Update distances
    for k in range(1000):
        distances[k,:], distance_vector[:,:,k] = get_dist(k)
    distances = distances + 0.00001

    neighbours = distances < r_cutoff


    for j in neighbours[0,:]:
        if j == False: continue
        force_new[:,j] += (4*epsilon*((sigma/distances[0,j])**12 - (sigma/distances[0,j])**6))*(distance_vector[:,0,j]/distances[0,j])
    
    vel += (force + force_new)*dt/2

    force = force_new

    abs_vel = np.linalg.norm(vel, axis=0)
    print(abs_vel.min(),abs_vel.max(),abs_vel.mean(),abs_vel.std())
            
    # # Plot velocity of particles
    # if(step%100==0):
    #     plt.hist(np.linalg.norm(vel, axis=0), bins=50)
    #     plt.xlim(0,1)
    #     plt.xlabel('Velocity')
    #     plt.ylabel('Count')
    #     plt.show() 
    print(step)  


0.0 164114286047116.5 1190859014584.4749 11790008253351.256
154.50635435248492 3.447319858672712e+48 1.4385957695833226e+46 1.837366339404402e+47
0
0.0 1.3605865387625158e+24 2.919308708569491e+21 6.093620632724955e+22
294.9356809302808 3.447319858672712e+48 1.4385957695833226e+46 1.837366339404402e+47
1


KeyboardInterrupt: 